# **CUSTOMER SEGMENT**

## **1.Import libary**

In [1]:
# data analysis libraries
import pandas as pd
import numpy as np
from datetime import datetime 

# data visualization libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# machine learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# remove warnings
import warnings
warnings.filterwarnings('ignore')



## **2.Reading Data**

In [2]:
#set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 15)

df_org = pd.read_csv("marketing_campaign.csv", sep="\t")
df_org.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


In [3]:
df_org.shape

(2240, 29)

In [4]:
df_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [5]:
# check missing values
df_org.isnull().sum()


ID                 0
Year_Birth         0
Education          0
Marital_Status     0
Income            24
                  ..
AcceptedCmp2       0
Complain           0
Z_CostContact      0
Z_Revenue          0
Response           0
Length: 29, dtype: int64

In [6]:
# check duplicates
df_org.duplicated().sum()

0

**=>** The DataFrame needs cleaning before feature engineering:
- The “Income” column has 24 missing values.
- The “Dt_Customer” column is stored as an object in “dd-mm-yyyy” format instead of datetime64[ns].
- The “Education” and “Marital_Status” columns are categorical.

### **Cleanning Data**

In [7]:
# make a copy of original dataframe
df = df_org.copy()

In [8]:
# drop rows na values in Income column
df = df.dropna(subset=['Income'])
df.isnull().sum()

ID                0
Year_Birth        0
Education         0
Marital_Status    0
Income            0
                 ..
AcceptedCmp2      0
Complain          0
Z_CostContact     0
Z_Revenue         0
Response          0
Length: 29, dtype: int64

In [9]:
#checking for categorial columns
cat_cols = df.select_dtypes(include=['object']).columns
for cat in cat_cols:
    print(f"\n Values count for {cat}: {df[cat].value_counts()}")
    print("-"*25)


 Values count for Education: Education
Graduation    1116
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: count, dtype: int64
-------------------------

 Values count for Marital_Status: Marital_Status
Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64
-------------------------

 Values count for Dt_Customer: Dt_Customer
31-08-2012    12
12-09-2012    11
14-02-2013    11
12-05-2014    11
20-08-2013    10
              ..
05-08-2012     1
18-11-2012     1
09-05-2014     1
26-06-2013     1
09-01-2014     1
Name: count, Length: 662, dtype: int64
-------------------------


In [10]:
# convert the Dt_Customer column to datetime format
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
df['Year'] = df['Dt_Customer'].dt.year


In [11]:
print ('Max date in dataset:', df['Dt_Customer'].max())
print ('Min date in dataset:', df['Dt_Customer'].min())

Max date in dataset: 2014-06-29 00:00:00
Min date in dataset: 2012-07-30 00:00:00


## **3. Feature engineering**

In [12]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
       'Year'],
      dtype='object')

=> I will create new features based on existing columns:

- Total_Spent = Sum of ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
- Total_Purchases = Sum of['NumWebPurchases', 'NumCatalogPurchases','NumStorePurchases', 'NumDealsPurchases']
- Living_Situation = alone / not_alone
- Education_Level = undergraduate / graduate / postgraduate
- Total_Children = Sum of ['Kidhome', 'Teenhome']
- Is_Parent = IF Total_Children > 0, Yes (1) ELSE No (0)
- Age = 2025 - Year_Birth
- Web_Ratio = ['NumWebPurchases'] / ['Total_Purchases']
- Store_Ratio = ['NumStorePurchases'] /['Total_Purchases']
- Deal_Ratio' = ['NumDealsPurchases'] / ['Total_Purchases']

In [13]:
# Calculate customer tenure
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
latest_date = df['Dt_Customer'].max()
df['Tenure'] = (latest_date - df['Dt_Customer']).dt.days

In [14]:
# Calculate Customer's Active Days until 2024-01-10

## Declare today's date
today_date_str = '2025-08-05'
today_date = pd.to_datetime(today_date_str, format= '%Y-%m-%d')

## Calculate 'Customer_Active_Days' 
df['Customer_Active_Days'] = (today_date - df['Dt_Customer']).dt.days

In [15]:
# Create new features
df['Total_Spent'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 
                                         'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

df['Total_Purchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 
                                             'NumStorePurchases', 'NumDealsPurchases']].sum(axis=1)

In [16]:
# Calculate channel preferences
df['Web_Ratio'] = df['NumWebPurchases'] / df['Total_Purchases'].replace(0, np.nan)
df['Store_Ratio'] = df['NumStorePurchases'] / df['Total_Purchases'].replace(0, np.nan)
df['Deal_Ratio'] = df['NumDealsPurchases'] / df['Total_Purchases'].replace(0, np.nan)

In [17]:
 #Living_Situation = alone / not_alone
df['Living_Situation'] = np.where(df['Marital_Status'].isin(['Single', 'Divorced', 'Widow', 'Alone', 'YOLO']), 
                                          'alone', 'not_alone')

In [18]:
# Education_Level = undergraduate / graduate / postgraduate
def map_education(edu):
    if edu in ['Basic', '2n Cycle']:
        return 'undergraduate'
    elif edu in ['Graduation']:
        return 'graduate'
    elif edu in ['Master', 'PhD']:
        return 'postgraduate'
    else:
        return 'unknown'


In [19]:
df['Education_Level'] = df['Education'].apply(map_education)


In [20]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Year,Tenure,Customer_Active_Days,Total_Spent,Total_Purchases,Web_Ratio,Store_Ratio,Deal_Ratio,Living_Situation,Education_Level
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,2012,663,4718,1617,25,0.320000,0.160000,0.120000,alone,graduate
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,2014,113,4168,27,6,0.166667,0.333333,0.333333,alone,graduate
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,2013,312,4367,776,21,0.380952,0.476190,0.047619,not_alone,graduate
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,2014,139,4194,53,8,0.250000,0.500000,0.250000,not_alone,graduate
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,2014,161,4216,422,19,0.263158,0.315789,0.263158,not_alone,postgraduate


In [21]:
# Total_Children = Sum of ['Kidhome', 'Teenhome']
df['Total_Children'] = df['Kidhome'] + df['Teenhome']

# Is_Parent = IF Total_Children > 0, Yes (1) ELSE No (0)
df['Is_Parent'] = np.where(df['Total_Children'] > 0, 1, 0)

# Age = 2025 - Year_Birth
df['Age'] = 2025 - df['Year_Birth']

In [22]:
# Handle infinite values
df = df.replace([np.inf, -np.inf], np.nan)


In [23]:
# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['Living_Situation', 'Education_Level'], drop_first=True)

In [24]:
print(df.columns.tolist())


['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response', 'Year', 'Tenure', 'Customer_Active_Days', 'Total_Spent', 'Total_Purchases', 'Web_Ratio', 'Store_Ratio', 'Deal_Ratio', 'Total_Children', 'Is_Parent', 'Age', 'Living_Situation_not_alone', 'Education_Level_postgraduate', 'Education_Level_undergraduate']


In [25]:
# Select features for clustering
features = ['Income', 'Age', 'Total_Spent', 'Total_Purchases', 'Tenure', 
            'Web_Ratio', 'Store_Ratio', 'Deal_Ratio', 'Is_Parent', 'Total_Children',
            'Living_Situation_not_alone', 'Education_Level_graduate', 
            'Education_Level_postgraduate', 'Education_Level_undergraduate']

In [27]:
cluster_data = df[features].copy()

KeyError: "['Education_Level_graduate'] not in index"

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
ID,2216.0,5588.353339,0.0,2814.75,5458.5,8421.75,11191.0,3249.376275
Year_Birth,2216.0,1968.820397,1893.0,1959.0,1970.0,1977.0,1996.0,11.985554
Income,2216.0,52247.251354,1730.0,35303.0,51381.5,68522.0,666666.0,25173.076661
Kidhome,2216.0,0.441787,0.0,0.0,0.0,1.0,2.0,0.536896
Teenhome,2216.0,0.505415,0.0,0.0,0.0,1.0,2.0,0.544181
...,...,...,...,...,...,...,...,...
Deal_Ratio,2212.0,0.180577,0.0,0.076923,0.166667,0.25,1.0,0.111044
Total_Children,2216.0,0.947202,0.0,0.0,1.0,1.0,3.0,0.749062
Is_Parent,2216.0,0.71435,0.0,0.0,1.0,1.0,1.0,0.451825
Age,2216.0,56.179603,29.0,48.0,55.0,66.0,132.0,11.985554


In [ ]:
cluster_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Income,2216.0,52247.251354,25173.076661,1730.0,35303.000000,51381.500000,68522.000000,666666.0
Age,2216.0,56.179603,11.985554,29.0,48.000000,55.000000,66.000000,132.0
Total_Spent,2216.0,607.075361,602.900476,5.0,69.000000,396.500000,1048.000000,2525.0
Total_Purchases,2216.0,14.880866,7.670957,0.0,8.000000,15.000000,21.000000,44.0
Tenure,2216.0,353.521209,202.434667,0.0,180.000000,355.500000,529.000000,699.0
Web_Ratio,2212.0,0.267265,0.098456,0.0,0.200000,0.260870,0.333333,1.0
Store_Ratio,2212.0,0.409738,0.118842,0.0,0.333333,0.411765,0.500000,1.0
Deal_Ratio,2212.0,0.180577,0.111044,0.0,0.076923,0.166667,0.250000,1.0
Is_Parent,2216.0,0.714350,0.451825,0.0,0.000000,1.000000,1.000000,1.0
Total_Children,2216.0,0.947202,0.749062,0.0,0.000000,1.000000,1.000000,3.0


In [ ]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data)

In [ ]:
plotting_cols = [ "Income", "Recency", "Customer_Active_Days", "Age", "Total_Spent", "Is_Parent"]

sns.pairplot(data = df[plotting_cols]
             , hue = 'Is_Parent'
             , palette= (['steelblue','darkorange'])
)
plt.show()